## 2. Introduction to Spatial Metrics I: Pairwise spatial analysis

This section will focus on methods for pairwise spatial analysis - comparing whether two populations are spatially correlated. There are a large number of spatial metrics which deal with this topic, many of which we have tutorials for (https://docs.muspan.co.uk/latest/tutorials.html). Here, we'll focus on the cross-pair correlation function (xPCF) and Moran's I / Getis-Ord.


In this section we will:
- Introduce the cross-pair correlation function
- Understand how changing the parameters of the cross-PCF affects the results
- Use the topographical correlation map to visualise hotspots of correlation within a domain
- Explore the effect of changing the domain boundary on the cross-PCF, and understand the importance of accurate domain boundaries
- Explore alternative methods for describing pairwise spatial interactions between populations

### The cross-pair correlation function: exploring spatial colocalisation

Load in the `Synthetic-Points-Architecture` dataset (https://docs.muspan.co.uk/latest/muspan.datasets.html#datasets). We'll start by visualising the cells.

In [ ]:
import muspan as ms

domain = ms.datasets.load_example_domain('Synthetic-Points-Architecture')

ms.visualise.visualise(domain, 'Celltype')

We'll analyse this region because there are some very distinctive spatial behaviours in it. In particular, some of the populations are spatially colocalised, some are excluded from one another, and some are spatially independent. Let's try and use the pair correlation function to find out which are which.

First, let's take a look at MuSpAn's cross pair correlation function function. Remember that we can always use the [MuSpAn docs](https://docs.muspan.co.uk/latest/generated/muspan.spatial_statistics.cross_pair_correlation_function.html#muspan.spatial_statistics.cross_pair_correlation_function) to check what arguments we can change in the function. For now, let's leave the default arguments as they are, and just specify two cell populations to compare.

In [ ]:
population_A = ('Celltype','A')
population_B = ('Celltype','B')

ms.spatial_statistics.cross_pair_correlation_function(domain, population_A, population_B, visualise_output=True)

The only optional argument we changed here was `visualise_output=True`, which is a handy option available in the majority of MuSpAn's analysis functions to easily see what the function output looks like. In this case, we can see that the default parameters have given us a nice looking curve: it is much bigger than 1 for small values of $r$, suggesting positive correlation between cell types A and B at short length scales, before dipping down.

Let's think about the interpretation of the cross-PCF. In particular, we'll think about the three key parameters `max_R=100`, `annulus_step=10`, and `annulus_width=10`. Conceptually, we interpret the cross-PCF by imagining an annulus of inner radius $r$ centred on a cell of type $A$. This annulus has width `annulus_width`, and the function that we consider here if calculated at specific values of $r$ (one every `annulus_step` - so here at 0, 10, 20, ..., 100).

We'll start by playing with those parameters. We'll also collect the output into two variables, $r$ and $g$, that we could use to plot the function (we won't for now, they're just here so that the arrays don't get printed out).

In [ ]:
r, g = ms.spatial_statistics.cross_pair_correlation_function(domain, population_A, population_B, 
                                                      annulus_step=5,
                                                      annulus_width=10,
                                                      max_R=250,
                                                      visualise_output=True)

Using a smaller annulus step gives a smoother curve, but notice that it took longer for the function to run this time. This is a common feature of spatial analysis; we have to make a trade off between computational complexity and the level of detail that we want to consider.

Plotting this curve shows some interesting bumps and dips. We have dips (local minima) around $r \approx 60, 125$ and peaks (local maxima) at $r \approx 100, 200$. How might you interpret these bumps?

Let's plot the cross-PCF for each possible combination of celltypes in this example, [using the code from the cross-PCF tutorial](https://docs.muspan.co.uk/latest/_collections/spatial_analysis_methods/Spatial%20stats%20-%201%20-%20pcf.html).

In [ ]:
import matplotlib.pyplot as plt
# Define the cell types to be analyzed
celltypes = ['A', 'B', 'C', 'D']

# Create a 4x4 subplot for visualizing the cross-PCF for each combination of cell types
fig, axes = plt.subplots(4, 4, figsize=(12, 12))

# Loop through each combination of cell types
for i in range(4):
    for j in range(4):
        # Calculate the cross-PCF for the current combination of cell types
        r, PCF = ms.spatial_statistics.cross_pair_correlation_function(
            domain,
            ('Celltype', celltypes[i]),
            ('Celltype', celltypes[j]),
            max_R=200,
            annulus_step=5,
            annulus_width=25
        )

        # Select the current subplot
        ax = axes[i, j]

        # Plot the cross-PCF
        ax.plot(r, PCF)

        # Add a horizontal line at y=1 to indicate the CSR baseline
        ax.axhline(1, color='k', linestyle=':')

        # Set the y-axis limit
        ax.set_ylim([0, 7])

        # Label the y-axis with the cross-PCF notation
        ax.set_ylabel(f'$g_{{{celltypes[i]}{celltypes[j]}}}(r)$')

        # Label the x-axis with the distance r
        ax.set_xlabel('$r$')

# Adjust the layout to prevent overlap
plt.tight_layout()

Which cell types are colocalised? Which are not?

In Figure 4 of https://doi.org/10.1038/s41467-023-42421-0, a similar grid of cross-PCF values is used to get a bird's eye perspective on which cell types are spatially correlated across regions of interest in covid lung tissue. There, the authors condense the full cross-PCFs into a single value, the measurement at $r=20$. Can you see the advantages (and disadvantages) of this choice?

### Identifying hotspots

If we've used pair correlation functions to identify that there is spatial correlation present between two cell types in a domain, a natural question to follow on with is: <i>where</i> in the domain is this corelation occurring? MuSpAn provides several methods to answer this question. Let's start by considering the [topographical correlation map](https://doi.org/10.1017/S2633903X24000011), a local indicator of spatial association that shows where in the domain cells of type B are positively, or negatively, associated with cells of type A. Let's take a look at this ([MuSpAn docs](https://docs.muspan.co.uk/latest/generated/muspan.spatial_statistics.topographical_correlation_map.html)).

In [ ]:
ms.spatial_statistics.topographical_correlation_map(domain, population_A, population_B, visualise_output=True)

The TCM works by calculating the local contribution to the PCF that each individual cell of type A makes, by considering the cumulative contribution of all annuli up to a radius $r$ around that cell <sub>(or more accurately, a normalised form the contribution of that cell to Ripley's K function)</sub>. The hotspots in this plot show us where cells of type B are strongly associated around cells of type A.

For another example, let's plot the association between cells of type D and cells of type A.

In [ ]:
ms.spatial_statistics.topographical_correlation_map(domain, ('Celltype','D'), ('Celltype','A'), visualise_output=True)

This TCM seems to show areas of strong positive, and negative, correlation between the two cell types. Let's plot the locations of cells of type A and D on top of the TCM.

In [ ]:
ms.spatial_statistics.topographical_correlation_map(domain, ('Celltype','D'), ('Celltype','A'), visualise_output=True)
ms.visualise.visualise(domain,'Celltype',objects_to_plot=('Celltype','A'),ax=plt.gca(),add_cbar=False)
ms.visualise.visualise(domain,'Celltype',objects_to_plot=('Celltype','D'),ax=plt.gca(),add_cbar=False)

Compare this to the plot of $g_{DA}$ above. What do you see? Do you think there is a real spatial association between cells of types D and A?

### Exploring further

1. A key part of the cross-PCF analysis, and indeed any spatial analysis, is the shape of the domain (i.e., the boundary). Load in any example domain, and calculate the cross-PCF between two cell types. Now [try manually changing the domain boundary](https://docs.muspan.co.uk/latest/muspan.domain.html#muspan.domain.estimate_boundary) to make it much bigger (for instance, twice as big in both the $x$ and $y$ directions). What happens when you recalculate the same cross-PCF on the same cell types? Can you explain why?

2. The boundary of a cross-PCF is extremely important, as we've just seen. Try [following the tutorial on using shapes as calculation boundaries for cross-PCFs](https://docs.muspan.co.uk/latest/_collections/spatial_analysis_methods/Spatial%20stats%20-%202%20-%20boundary%20corrections.html).

3. Load in the dataset `Synthetic-Points-Exclusion` and calculate the cross-PCF betwen cell types A and B. You should see strong spatial correlation, as the points are all in the same part of the domain. Try [following the instructions in the tutorial on converting points to shapes](https://docs.muspan.co.uk/latest/_collections/working_with_objects/WWO%20-%203%20-%20points%20to%20shape%203.html) to make the convex hull around all points of type D. Can you use this shape as the boundary for the cross-PCF analysis (between cell types A and B)? What do you see?

4. There are many other approaches for comparing pairs of point populations, many of which can be found in [the documentation for MuSpAn's spatial statistics](https://docs.muspan.co.uk/latest/muspan.spatial_statistics.html). Try out some others - you may want to consider the K-function, the J-function, and the average nearest neighbour index (for example).

5. The TCM provides one way of showing local correlation between cells of different types, but there are many other methods. Try following the tutorial on Getis-Ord hotspot analysis to identify whether there is a correlation between cells of types D and A in the `Synthetic-Points-Architecture` dataset.